## Import Libraries

In [1]:
import numpy as np
import pandas as pd
import scipy.optimize as optimize
import matplotlib.pyplot as plt

In [2]:
# A = Original matrix
# B = Square root of Matrix A
# U, V, L = Matrices obtained after SVD
# Ut, Vt, Lt = Truncated U, V, L matrices
# Bnp = Matrix obtained from U, V, L with no phase added or truncation done
# Bp = Matrix obtained from U, V, L after truncation and phase added
# Bt = Matrix obtained from U, V, L after truncation, no phase added

## Optimization Algorithm

In [3]:
def newMat_2(x):
    l0,l1 = Lt[0], Lt[1]
    U_new = np.zeros((Ut.shape), dtype=np.cfloat)
    for i in range(Ut.shape[0]):
      U_new[i][0] = np.cos(x[i]) / l0
      U_new[i][1] = (np.sin(x[i]) / l1) * np.exp(1j*x[i+Ut.shape[0]])
    return np.dot(np.dot(U_new,np.diag(Lt)), Vt)


def newMat_3(x):
    l0,l1,l2 = Lt[0], Lt[1], Lt[2]
    U_new = np.zeros((Ut.shape), dtype=np.cfloat)
    for i in range(Ut.shape[0]):
      U_new[i][0] = np.cos(x[i]) / l0
      U_new[i][1] = (np.sin(x[i]) / l1) * (np.cos(x[i+Ut.shape[0]]) / l1) * np.exp(1j*x[i+2*Ut.shape[0]])
      U_new[i][2] = (np.sin(x[i]) / l2) * (np.sin(x[i+Ut.shape[0]]) / l2) * np.exp(1j*x[i+3*Ut.shape[0]])
    return np.dot(np.dot(U_new,np.diag(Lt)), Vt)


def newMat_4(x):
    l0,l1,l2,l3 = Lt[0], Lt[1], Lt[2], Lt[3]
    U_new = np.zeros((Ut.shape), dtype=np.cfloat)
    for i in range(Ut.shape[0]):
      U_new[i][0] = (np.cos(x[i]) / l0) * (np.cos(x[i+Ut.shape[0]]) / l0)
      U_new[i][1] = (np.cos(x[i]) / l1) * (np.sin(x[i+Ut.shape[0]]) / l1) * np.exp(1j*x[i+2*Ut.shape[0]])
      U_new[i][2] = (np.sin(x[i]) / l2) * (np.cos(x[i+Ut.shape[0]]) / l2) * np.exp(1j*x[i+3*Ut.shape[0]])
      U_new[i][3] = (np.sin(x[i]) / l3) * (np.sin(x[i+Ut.shape[0]]) / l3) * np.exp(1j*x[i+4*Ut.shape[0]])
    return np.dot(np.dot(U_new,np.diag(Lt)), Vt)


def costFn(x):
    Bp = newMat_4(x)
    loss = np.linalg.norm(B**2 - np.abs(Bp)**2)
    return (loss)

In [4]:
final_arr = pd.DataFrame(columns=["m", "n", 
                                  "mean_initial_dist", "mean_final_dist", "mean_RI", "std_RI"])

m,n=6,7
k=4
print("k = ",k," m = ",m,", n = ",n)
res = np.zeros((10,2))
for i in range(10):
    A = np.random.rand(m, n)
    for j in range(m): A[j] /= sum(A[j])
    B = np.sqrt(A)
    U, L, V = np.linalg.svd(B, full_matrices=False)
    Bnp = np.dot(np.dot(U,np.diag(L)), V)
    Ut = U[:, :k]
    Vt = V[:k]
    Lt = L[:k]
    Bt = np.dot(np.dot(Ut,np.diag(Lt)), Vt)
    initial_guess = np.ones((2*m*(k-1),), dtype=np.longdouble)
    result = optimize.minimize(fun=costFn, x0=initial_guess, tol=1e-10, method='Nelder-Mead', options={'maxiter':1e+10})
    res[i][0] = (np.linalg.norm(B**2 - Bt**2))
    res[i][1] = costFn(result.x)
    if ((i+1)%1==0):
      print(i+1, " ", end='')
mean_initial, mean_final = res.mean(axis=0)
ri_mean= ((mean_initial - mean_final)*100/mean_initial)
ri_std = np.mean(np.std(res[:,0]-res[:,1]))
data = {"m": m, "n": n, 
        "mean_initial_dist": mean_initial, "mean_final_dist": mean_final,
        "mean_RI": ri_mean, "std_RI": ri_std}
final_arr = final_arr.append(data, ignore_index=True)
print("\n")

k =  4  m =  6 , n =  7
1  2  3  4  5  6  7  8  9  10  



In [5]:
np.column_stack((res, res[:,0]-res[:,1]))

array([[ 0.12701459,  0.46863408, -0.34161949],
       [ 0.14439474,  0.30686331, -0.16246857],
       [ 0.16666009,  0.49919862, -0.33253853],
       [ 0.09336854,  0.26704338, -0.17367484],
       [ 0.1123745 ,  0.37972093, -0.26734643],
       [ 0.13977655,  0.13350189,  0.00627466],
       [ 0.08397393,  0.24253262, -0.1585587 ],
       [ 0.10868955,  0.249787  , -0.14109745],
       [ 0.12900818,  0.17519114, -0.04618296],
       [ 0.14207586,  0.46557105, -0.32349519]])

In [6]:
final_arr

,m,n,mean_initial_dist,mean_final_dist,mean_RI,std_RI
0,6.0,7.0,0.124734,0.318804,-155.588124,0.114261
